In [1]:
import os
import sys
import pandas as pd
import numpy as np

seed = 16
np.random.seed(seed)

from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


c:\users\garrick\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#check using system GPU for processing

from tensorflow.python.client import device_lib
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1475249643691004847
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1489587404
locality {
  bus_id: 1
}
incarnation: 2680521960539963153
physical_device_desc: "device: 0, name: GeForce GTX 860M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [3]:
# declare ImageDataGenerator and flow_from_directory vars
batch_size=10
num_classes = 120

In [4]:
# changing directory for flow_from_directory method
os.chdir('C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets')

In [5]:
#obtain train and test labels
from scipy.io import loadmat

y_train_rough = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_list.mat''')['labels']

y_test = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\test_list.mat''')['labels']

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
labels = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_list.mat''')['labels']
files = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_list.mat''')['file_list']
labels = [item for label in labels for item in label] #this is flattening a list of lists, because for some reason ever label is stored as a list
files = [item for file in files for item in file]
df = pd.DataFrame({'labels':labels, 'files':files})
train, validate = train_test_split(df, test_size = 0.2, stratify=labels)


In [9]:
X_train = train['files']
y_train = to_categorical(train['labels'])
X_val = validate['files']
y_val = to_categorical(validate['labels'])

In [10]:
train_datagen = ImageDataGenerator(rotation_range=15, shear_range=0.1, channel_shift_range=20,
                                    width_shift_range=0.1,  height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True,
                                    fill_mode='nearest', rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory('train', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)

validation_generator = validation_datagen.flow_from_directory('validation', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)


test_generator = test_datagen.flow_from_directory('test', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)


Found 9600 images belonging to 120 classes.
Found 2400 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [11]:
train_generator

In [24]:
#obtain validation labels
# no longer used, utilizing sklearn's train_test_split function

'''
y_validation = []
y_train = []

for i in range(120):
    i = i*100
    
    #begin_index = train_full_list[i::100]
    #end_index = train_full_list[i+100::100]
    
    slice = y_train_rough[i:i+100,]
    y_train.append(slice[:80])
    y_validation.append(slice[80:])

y_train = np.concatenate(y_train, axis=0)
y_validation = np.concatenate(y_validation, axis=0)
'''

'\ny_validation = []\ny_train = []\n\nfor i in range(120):\n    i = i*100\n    \n    #begin_index = train_full_list[i::100]\n    #end_index = train_full_list[i+100::100]\n    \n    slice = y_train_rough[i:i+100,]\n    y_train.append(slice[:80])\n    y_validation.append(slice[80:])\n\ny_train = np.concatenate(y_train, axis=0)\ny_validation = np.concatenate(y_validation, axis=0)\n'

In [12]:
# using a simple CNN to start

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import keras.utils
from keras import backend as K
K.set_image_dim_ordering('tf')


In [14]:
input_shape = (224,224, 3)

# create the model

model = Sequential()
model.add(Conv2D(64, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
     

# Compile model
epochs = 25
lrate = 0.003
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 54, 54, 64)        23296     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 32)        32800     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 64)          32832     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
__________

In [15]:
# configure tensorflow before fitting model
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [ ]:
# train/fit the simple CNN using flow from directory **********CURRENTLY CRASHING********

model.fit_generator(train_generator,
                    steps_per_epoch=len(X_train) / 32, epochs=epochs)

Epoch 1/25


(12000, 12000)

In [ ]:
from keras.applications import VGG16
 
#Load the VGG model
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=input_shape) 

In [ ]:
ImageDataGenerator?